To use the MPC's STAC API we will need the `Clinet` function from the `pystac_client` library and the `planetary_computer` library:

In [74]:
import numpy as np
import geopandas as gpd
import rioxarray as rioxr
import matplotlib.pyplot as plt

from shapely.geometry import shape

# used to access STAC catalogs
from pystac_client import Client
# use to sign items from the MPC STAC catalog
import planetary_computer

# ----- other libraries for nice ouputs
import rich.table
from IPython.display import Image

In [3]:
# access catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [7]:
# metadata from the catalog
#print('ID:', catalog.id)
print('Title:', catalog.title)
print('Description:', catalog.description)

Title: Microsoft Planetary Computer STAC API
Description: Searchable spatiotemporal metadata describing Earth science datasets hosted by the Microsoft Planetary Computer


In [10]:
catalog.get_collections()

<generator object Client.get_collections at 0x7f18e5784040>

In [13]:
collections = list(catalog.get_collections())
collections

[<CollectionClient id=daymet-annual-pr>,
 <CollectionClient id=daymet-daily-hi>,
 <CollectionClient id=3dep-seamless>,
 <CollectionClient id=3dep-lidar-dsm>,
 <CollectionClient id=fia>,
 <CollectionClient id=sentinel-1-rtc>,
 <CollectionClient id=gridmet>,
 <CollectionClient id=daymet-annual-na>,
 <CollectionClient id=daymet-monthly-na>,
 <CollectionClient id=daymet-annual-hi>,
 <CollectionClient id=daymet-monthly-hi>,
 <CollectionClient id=daymet-monthly-pr>,
 <CollectionClient id=gnatsgo-tables>,
 <CollectionClient id=hgb>,
 <CollectionClient id=cop-dem-glo-30>,
 <CollectionClient id=cop-dem-glo-90>,
 <CollectionClient id=goes-cmi>,
 <CollectionClient id=terraclimate>,
 <CollectionClient id=nasa-nex-gddp-cmip6>,
 <CollectionClient id=gpm-imerg-hhr>,
 <CollectionClient id=gnatsgo-rasters>,
 <CollectionClient id=3dep-lidar-hag>,
 <CollectionClient id=3dep-lidar-intensity>,
 <CollectionClient id=3dep-lidar-pointsourceid>,
 <CollectionClient id=mtbs>,
 <CollectionClient id=noaa-c-cap>,
 

In [14]:
print('Number of collections:', len(collections))
print("Collections IDs:")
for collection in collections:
    print('-', collection.id)

Number of collections: 122
Collections IDs:
- daymet-annual-pr
- daymet-daily-hi
- 3dep-seamless
- 3dep-lidar-dsm
- fia
- sentinel-1-rtc
- gridmet
- daymet-annual-na
- daymet-monthly-na
- daymet-annual-hi
- daymet-monthly-hi
- daymet-monthly-pr
- gnatsgo-tables
- hgb
- cop-dem-glo-30
- cop-dem-glo-90
- goes-cmi
- terraclimate
- nasa-nex-gddp-cmip6
- gpm-imerg-hhr
- gnatsgo-rasters
- 3dep-lidar-hag
- 3dep-lidar-intensity
- 3dep-lidar-pointsourceid
- mtbs
- noaa-c-cap
- 3dep-lidar-copc
- modis-64A1-061
- alos-fnf-mosaic
- 3dep-lidar-returns
- mobi
- landsat-c2-l2
- era5-pds
- chloris-biomass
- kaza-hydroforecast
- planet-nicfi-analytic
- modis-17A2H-061
- modis-11A2-061
- daymet-daily-pr
- 3dep-lidar-dtm-native
- 3dep-lidar-classification
- 3dep-lidar-dtm
- gap
- modis-17A2HGF-061
- planet-nicfi-visual
- gbif
- modis-17A3HGF-061
- modis-09A1-061
- alos-dem
- alos-palsar-mosaic
- deltares-water-availability
- modis-16A3GF-061
- modis-21A2-061
- us-census
- jrc-gsw
- deltares-floods
- modi

In [17]:
naip_collection = catalog.get_child('naip')
naip_collection

id: naip
title: NAIP: National Agriculture Imagery Program
"description: The [National Agriculture Imagery Program](https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/) (NAIP) provides U.S.-wide, high-resolution aerial imagery, with four spectral bands (R, G, B, IR). NAIP is administered by the [Aerial Field Photography Office](https://www.fsa.usda.gov/programs-and-services/aerial-photography/) (AFPO) within the [US Department of Agriculture](https://www.usda.gov/) (USDA). Data are captured at least once every three years for each state. This dataset represents NAIP data from 2010-present, in [cloud-optimized GeoTIFF](https://www.cogeo.org/) format."
"providers: USDA Farm Service Agency (producer, licensor) Esri (processor) Microsoft (host, processor)"
type: Collection
"item_assets: {'image': {'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data'], 'title': 'RGBIR COG tile', 'eo:bands': [{'name': 'Red', 'common_name': 'red'}, {'name': 'Green', 'common_name': 'green'}, {'name': 'Blue', 'common_name': 'blue'}, {'name': 'NIR', 'common_name': 'nir', 'description': 'near-infrared'}]}, 'metadata': {'type': 'text/plain', 'roles': ['metadata'], 'title': 'FGDC Metdata'}, 'thumbnail': {'type': 'image/jpeg', 'roles': ['thumbnail'], 'title': 'Thumbnail'}}"
msft:region: westeurope
msft:container: naip
msft:storage_account: naipeuwest
"msft:short_description: NAIP provides US-wide, high-resolution aerial imagery. This dataset includes NAIP images from 2010 to the present."
https://stac-extensions.github.io/item-assets/v1.0.0/schema.json


How to do a search in the catalogue by collection, time, and region. 

In [32]:
# Temporal range of interest
time_range = "2018-01-01/2023-01-01"

# NCEAS bounding box (as a GeoJSON)
bbox = {
    "type": "Polygon",
    "coordinates":[
        [
            [-119.70608227128903, 34.426300194372274],
            [-119.70608227128903, 34.42041139020533],
            [-119.6967885126002, 34.42041139020533],
            [-119.6967885126002, 34.426300194372274],
            [-119.70608227128903, 34.426300194372274]
        ]
    ],
}

In [33]:
search = catalog.search(
    collections=['naip'],
    intersects=bbox,
    datetime=time_range)

In [34]:
items = search.item_collection()
len(items)

2

https://pystac.readthedocs.io/en/stable/api/item_collection.html#pystac-item-collection

Each `pystac.Item <https://pystac.readthedocs.io/en/stable/api/pystac.html#pystac.Item>`__ in this ItemCollection includes all the metadata for that scene. STAC Items are GeoJSON features, and so can be loaded by libraries like geopandas.

In [37]:
type(items)

pystac.item_collection.ItemCollection

In [38]:
items

id: ca_m_3411935_sw_11_060_20200521
"bbox: [-119.754272, 34.371741, -119.683292, 34.440724]"
gsd: 0.6
datetime: 2020-05-21T00:00:00Z
naip:year: 2020
"proj:bbox: [246930.0, 3806808.0, 253260.0, 3814296.0]"
proj:epsg: 26911
naip:state: ca
"proj:shape: [12480, 10550]"
"proj:transform: [0.6, 0.0, 246930.0, 0.0, -0.6, 3814296.0, 0.0, 0.0, 1.0]"
https://stac-extensions.github.io/eo/v1.0.0/schema.json


In [68]:
item = items[0]
type(item)

pystac.item.Item

In [43]:
# table = rich.table.Table("Asset Key", "Description")

# for asset_key, asset in item.assets.items():
#     table.add_row(asset_key, asset.title)
# table

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key        ┃ Description                     ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image            │ RGBIR COG tile                  │
│ thumbnail        │ Thumbnail                       │
│ tilejson         │ TileJSON with default rendering │
│ rendered_preview │ Rendered preview                │
└──────────────────┴─────────────────────────────────┘

In [67]:
print('id:' , item.id)
#print('geometry:', item.geometry)

id: ca_m_3411935_sw_11_060_20200521


In [ ]:
item.properties

{'gsd': 0.6,
 'datetime': '2020-05-21T00:00:00Z',
 'naip:year': '2020',
 'proj:bbox': [246930.0, 3806808.0, 253260.0, 3814296.0],
 'proj:epsg': 26911,
 'naip:state': 'ca',
 'proj:shape': [12480, 10550],
 'proj:transform': [0.6, 0.0, 246930.0, 0.0, -0.6, 3814296.0, 0.0, 0.0, 1.0]}

Assets actually have the data

In [63]:
for key in item.assets.keys():
    print(key, '--', item.assets[key].title)

image -- RGBIR COG tile
thumbnail -- Thumbnail
tilejson -- TileJSON with default rendering
rendered_preview -- Rendered preview


In [53]:
# sb_naip_items = gpd.GeoDataFrame.from_features(items.to_dict(), 
#                                                crs='epsg:4326')
# sb_naip_items

In [70]:
item.assets['rendered_preview'].href

'https://planetarycomputer.microsoft.com/api/data/v1/item/preview.png?collection=naip&item=ca_m_3411935_sw_11_060_20200521&assets=image&asset_bidx=image%7C1%2C2%2C3&format=png'

In [75]:
Image(url=item.assets["rendered_preview"].href, width=500)

The actual data assets are in private Azure Blob Storage containers. If forget to pass modifier=planetary_computer.sign_inplace or manually sign the item, then you’ll get a 404 when trying to access the asset.

In [76]:
# if we do not 
item.assets['image'].href

'https://naipeuwest.blob.core.windows.net/naip/v002/ca/2020/ca_060cm_2020/34119/m_3411935_sw_11_060_20200521.tif?st=2023-11-09T21%3A47%3A25Z&se=2023-11-17T21%3A47%3A25Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2023-11-10T21%3A47%3A24Z&ske=2023-11-17T21%3A47%3A24Z&sks=b&skv=2021-06-08&sig=G/ddLlxJK22h/C2N01bnh1zkJ26NFZ3a8lvbm5oTl54%3D'

In [77]:
rioxr.open_rasterio(item.assets['image'].href)

<xarray.DataArray (band: 4, y: 12480, x: 10550)>
[526656000 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1 2 3 4
  * x            (x) float64 2.469e+05 2.469e+05 ... 2.533e+05 2.533e+05
  * y            (y) float64 3.814e+06 3.814e+06 ... 3.807e+06 3.807e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:             Area
    TIFFTAG_IMAGEDESCRIPTION:  OrthoVista
    TIFFTAG_RESOLUTIONUNIT:    1 (unitless)
    TIFFTAG_SOFTWARE:          Trimble Germany GmbH
    TIFFTAG_XRESOLUTION:       1
    TIFFTAG_YRESOLUTION:       1
    _FillValue:                0
    scale_factor:              1.0
    add_offset:                0.0